In [ ]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
import os


c:\Users\nnnur\Desktop\хакатон\faiss_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Загружаем эмбеддинг-модель
model = SentenceTransformer(R'C:\Users\nnnur\Desktop\хакатон\paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 


# База знаний (фразы-суфлёры для оператора)
knowledge_base = [
    # Приветствия и стандартные фразы
    "Добрый день! Чем могу помочь?",
    "Здравствуйте! Слушаю вас.",
    "Рады вас слышать! Какой у вас вопрос?",
    "Приветствуем в нашем сервисе! Чем могу быть полезен?",
    
    # Уточняющие вопросы
    "Пожалуйста, уточните номер договора.",
    "Для помощи мне нужно узнать ваш идентификатор клиента.",
    "Не могли бы вы уточнить детали вашего запроса?",
    "Какой именно сервис вас интересует?",
    "Вы обращаетесь по вопросу кредита, депозита или другого продукта?",
    
    # Оплата и платежи
    "Вы можете оплатить через мобильное приложение или терминал.",
    "Для оплаты доступны онлайн-банкинг, мобильное приложение и платежные терминалы.",
    "Оплатить можно через личный кабинет на нашем сайте.",
    "Реквизиты для оплаты будут отправлены вам в SMS.",
    "Автоплатеж можно настроить в личном кабинете.",
    "При оплате через терминал комиссия не взимается.",
    
    # Кредиты и задолженности
    "Мы можем предложить вам реструктуризацию задолженности.",
    "Информацию о текущей задолженности можно узнать в личном кабинете.",
    "График платежей был отправлен вам на email при оформлении кредита.",
    "Вы можете подать заявку на увеличение кредитного лимита.",
    "Досрочное погашение кредита доступно без комиссий.",
    
    # Безопасность и верификация
    "Для безопасности мне нужно провести верификацию.",
    "Ожидайте СМС с кодом подтверждения.",
    "К сожалению, я не могу разглашать эту информацию без верификации.",
    "Пожалуйста, назовите кодовое слово из договора.",
    "Доступ к этой информации возможен только после подтверждения личности.",
    
    # Технические проблемы
    "Приносим извинения за технические неполадки. Мы уже работаем над решением.",
    "Попробуйте обновить страницу или зайти позже.",
    "Для решения проблемы попробуйте очистить кэш браузера.",
    "Наши технические специалисты уже в курсе проблемы.",
    
    # Переключение на специалиста
    "Перевожу вас на специалиста по данному вопросу.",
    "Мой коллега лучше поможет вам с этим вопросом.",
    "Соединяю с отделом по работе с клиентами.",
    "Этот вопрос требует уточнения, перевожу на профильного специалиста.",
    
    # Завершение разговора
    "Спасибо за обращение, хорошего дня!",
    "Благодарим за ваш звонок!",
    "Если у вас будут еще вопросы - мы всегда на связи!",
    "Желаем вам приятного дня!",
    "Всего доброго!",
    
    # Информация о продуктах
    "У нас есть специальные условия для новых клиентов.",
    "Подробнее о тарифах можно узнать на нашем сайте.",
    "Актуальные акции отражены в личном кабинете.",
    "Вы можете оформить продукт онлайн без визита в офис.",
    
    # Работа с претензиями
    "Ваша жалоба зарегистрирована под номером #XXXX.",
    "Мы рассмотрим ваше обращение в течение 3 рабочих дней.",
    "Приносим извинения за доставленные неудобства.",
    "Ваше обращение будет передано в службу контроля качества.",
    
    # Карты и счета
    "Информация о вашей карте доступна в мобильном приложении.",
    "Вы можете заказать перевыпуск карты в личном кабинете.",
    "Лимиты по карте можно увеличить, предоставив справку о доходах.",
    "Выписку по счету можно получить в отделении или онлайн.",
    
    # Мобильное приложение
    "Мобильное приложение доступно в AppStore и Google Play.",
    "В приложении доступны все основные операции.",
    "Вы можете настроить уведомления в настройках приложения.",
    "Для входа в приложение используйте те же данные, что и в личном кабинете.",
    
    # Валютные операции
    "Актуальные курсы валют отображаются в личном кабинете.",
    "Вы можете зарезервировать валюту онлайн и забрать в отделении.",
    "Лимит на валютные операции составляет $10,000 в месяц.",
    
    # Страховые продукты
    "Мы можем предложить вам несколько вариантов страхования.",
    "Страховой полис можно оформить онлайн за 5 минут.",
    "У нас есть специальные условия по страхованию для держателей карт.",
    
    # Инвестиции
    "Наши консультанты помогут вам подобрать инвестиционный продукт.",
    "Вы можете начать инвестировать с суммы от 1000 рублей.",
    "Доходность за прошлый год составила 7.2% годовых."
]

# Преобразуем фразы в эмбеддинги
kb_embeddings = model.encode(knowledge_base)

# Строим FAISS-индекс
dimension = kb_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(kb_embeddings))

# Входящий запрос клиента
client_input = "Как я могу оплатить кредит?"

# Кодируем запрос
query_embedding = model.encode([client_input])

# Поиск наиболее близкого ответа
top_k = 3
D, I = index.search(np.array(query_embedding), top_k)


# Печатаем наиболее подходящие реплики
print("🔍 Подходящие подсказки оператору:")
for idx in I[0]:
    print("-", knowledge_base[idx])

print(D)


🔍 Подходящие подсказки оператору:
- График платежей был отправлен вам на email при оформлении кредита.
- Досрочное погашение кредита доступно без комиссий.
- Вы обращаетесь по вопросу кредита, депозита или другого продукта?
[[15.761235 16.273132 18.275688]]


In [ ]:
# Сохраняем FAISS-индекс
faiss.write_index(index, "faiss_index.bin")

# Сохраняем текстовую базу знаний
with open("knowledge_base.pkl", "wb") as f:
    pickle.dump(knowledge_base, f)

In [ ]:
# Загружаем FAISS-индекс
index = faiss.read_index("faiss_index.bin")

# Загружаем базу знаний
with open("knowledge_base.pkl", "rb") as f:
    knowledge_base = pickle.load(f)

In [6]:
# Новая фраза
new_phrase = "Вы можете оплатить кредит через наш сайт или терминал."

# 1. Добавляем в список
knowledge_base.append(new_phrase)

# 2. Получаем эмбеддинг
new_embedding = model.encode([new_phrase])

# 3. Добавляем в FAISS-индекс
index.add(np.array(new_embedding))

# 4. (опционально) Сохраняем обновлённую базу
faiss.write_index(index, "faiss_index.bin")
with open("knowledge_base.pkl", "wb") as f:
    pickle.dump(knowledge_base, f)

In [ ]:
def find_similar_responses(user_input, model, index, knowledge_base, top_k=3):
    embedding = model.encode([user_input])
    D, I = index.search(np.array(embedding), top_k)
    return [knowledge_base[i] for i in I[0]]


results = find_similar_responses("Как оплатить долг?", model, index, knowledge_base)
print("🔍 Подсказки:")
for r in results:
    print("-", r)

In [ ]:
class AISuggester:
    def __init__(self, model_path, faiss_index_path="faiss_index.bin", kb_path="knowledge_base.pkl"):
        self.model = SentenceTransformer(model_path)
        self.faiss_index_path = faiss_index_path
        self.kb_path = kb_path
        self.knowledge_base = []
        self.index = None
        self._load_or_init()

    def _load_or_init(self):
        if os.path.exists(self.kb_path) and os.path.exists(self.faiss_index_path):
            self.load()
        else:
            self.index = faiss.IndexFlatL2(self.model.get_sentence_embedding_dimension())
            self.knowledge_base = []

    def add_phrase(self, phrase: str):
        embedding = self.model.encode([phrase])
        self.index.add(np.array(embedding))
        self.knowledge_base.append(phrase)

    def add_phrases(self, phrases: list):
        embeddings = self.model.encode(phrases)
        self.index.add(np.array(embeddings))
        self.knowledge_base.extend(phrases)

    def search(self, query: str, top_k: int = 3):
        if not self.knowledge_base:
            return []
        query_embedding = self.model.encode([query])
        D, I = self.index.search(np.array(query_embedding), top_k)
        return [self.knowledge_base[i] for i in I[0]]

    def save(self):
        faiss.write_index(self.index, self.faiss_index_path)
        with open(self.kb_path, "wb") as f:
            pickle.dump(self.knowledge_base, f)

    def load(self):
        self.index = faiss.read_index(self.faiss_index_path)
        with open(self.kb_path, "rb") as f:
            self.knowledge_base = pickle.load(f)


In [10]:
# 1. Инициализация
suggester = AISuggester(
    model_path=r"C:\Users\nnnur\Desktop\хакатон\paraphrase-multilingual-MiniLM-L12-v2"
)

# 2. Добавление базы знаний (однократно)
initial_phrases = [
    "Добрый день! Чем могу помочь?",
    "Пожалуйста, уточните номер договора.",
    "Оплатить можно через личный кабинет на нашем сайте.",
    "Для безопасности мне нужно провести верификацию.",
    "Спасибо за обращение, хорошего дня!"
]
suggester.add_phrases(initial_phrases)
suggester.save()

# 3. Загрузка при следующем запуске
# suggester = AISuggester(...)  # автоматом загрузит сохранённые данные

# 4. Поиск
results = suggester.search("Как оплатить кредит?")
for r in results:
    print("👉", r)


👉 График платежей был отправлен вам на email при оформлении кредита.
👉 Вы можете оплатить кредит через наш сайт или терминал.
👉 Вы обращаетесь по вопросу кредита, депозита или другого продукта?
